# ***NECESARIO***

In [1]:
!pip install TTS -qqq
!!pip install pydub -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 20.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 24.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 103.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.2 MB/

[]

# ***NECESARIO***

In [2]:
from TTS.api import TTS
from IPython.display import Audio
from pydub import AudioSegment
import os

# ***Opcional***

In [3]:
Audio("/content/fer.wav")

# ***NECESARIO***

In [4]:
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=True)

/usr/local/lib/python3.10/dist-packages/TTS/api.py:70: UserWarning: `gpu` will be deprecated. Please use `tts.to(device)` instead.
  warnings.warn("`gpu` will be deprecated. Please use `tts.to(device)` instead.")


 > You must confirm the following:
 | > "I have purchased a commercial license from Coqui: licensing@coqui.ai"
 | > "Otherwise, I agree to the terms of the non-commercial CPML: https://coqui.ai/cpml" - [y/n]
 | | > y
 > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v2


100%|█████████▉| 1.87G/1.87G [00:25<00:00, 74.3MiB/s]
100%|██████████| 1.87G/1.87G [00:25<00:00, 72.0MiB/s]
100%|██████████| 4.37k/4.37k [00:00<00:00, 21.8kiB/s]

100%|██████████| 361k/361k [00:00<00:00, 1.36MiB/s]
100%|██████████| 32.0/32.0 [00:00<00:00, 109iB/s]
 51%|█████▏    | 3.98M/7.75M [00:00<00:00, 22.0MiB/s]

 > Model's license - CPML
 > Check https://coqui.ai/cpml.txt for more info.
 > Using model: xtts


100%|██████████| 7.75M/7.75M [00:13<00:00, 22.0MiB/s]/usr/local/lib/python3.10/dist-packages/TTS/tts/layers/xtts/xtts_manager.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featur

# ***NO NECESARIO***

In [5]:
def read_and_split_text(text, max_tokens):
    # Dividir en palabras y luego en segmentos que no excedan max_tokens
    words = text.split()
    segments = []
    current_segment = []

    for word in words:
        # Simular la adición de la palabra al segmento y verificar el límite de tokens
        test_segment = current_segment + [word]
        if len(' '.join(test_segment).split()) > max_tokens:
            # Cuando se alcanza el límite, guardar el segmento actual y comenzar uno nuevo
            segments.append(' '.join(current_segment))
            current_segment = [word]
        else:
            current_segment = test_segment

    # Añadir el último segmento si contiene palabras
    if current_segment:
        segments.append(' '.join(current_segment))

    return segments

def synthesize_segments(segments, speaker_wav, language, output_prefix):
    # Convertir los segmentos a audio
    file_paths = []
    for i, segment in enumerate(segments):
        file_path = f"{output_prefix}_part_{i}.wav"
        tts.tts_to_file(text=segment, file_path=file_path, speaker_wav=speaker_wav, language=language)
        file_paths.append(file_path)
    return file_paths

def combine_audio(segments, output_file, output_prefix):
    combined = AudioSegment.silent(duration=0)
    for i in range(len(segments)):
        audio = AudioSegment.from_wav(f"{output_prefix}_part_{i}.wav")
        combined += audio
    combined.export(output_file, format='wav')

def clean_up(file_paths):
    # Elimina los archivos de audio individuales
    for file_path in file_paths:
        os.remove(file_path)

# Uso del código
prompt = '''
Había una vez en un valle rodeado de montañas, donde vivía un niño llamado Leo. Leo era muy curioso y amaba explorar el entorno natural que lo rodeaba. Un día, mientras estaba jugando en el bosque, se encontró con una pequeña mariposa que se había quedado atrapada en una hoja de trébol. Leo decidió ayudarla y liberó a la mariposa, pero mientras lo hacía, notó que la hoja de trébol estaba muy frágil y se doblaba bajo el peso del trébol.

"¿Qué está sucediendo?" preguntó Leo a la mariposa. "¿Por qué la hoja de trébol está tan frágil?"

La mariposa le explicó que el trébol era un ejemplo del problema del rebasamiento climático. "Los gases de efecto invernadero, como el dióxido de carbono y el metano, están causados por el hombre, y estos gases contribuyen a la estrecha capa de ozono que protege la Tierra de los radiación ultravioleta del sol", le explicó.
'''
speaker_wav = "/content/fer.wav"
language = "es"
output_prefix = "output"
final_output = "output.wav"
max_tokens = 30 #evitar warning

texto = prompt
segments = read_and_split_text(texto, max_tokens)
file_paths = synthesize_segments(segments, speaker_wav, language, output_prefix)
combine_audio(segments, final_output, output_prefix)
clean_up(file_paths)  # Llama a la función para limpiar los archivos temporales

 > Text splitted to sentences.
['Había una vez en un valle rodeado de montañas, donde vivía un niño llamado Leo.', 'Leo era muy curioso y amaba explorar el entorno natural que lo rodeaba.', 'Un día,']


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


 > Processing time: 9.145800352096558
 > Real-time factor: 0.6040185992348238
 > Text splitted to sentences.
['mientras estaba jugando en el bosque, se encontró con una pequeña mariposa que se había quedado atrapada en una hoja de trébol.', 'Leo decidió ayudarla y liberó a la mariposa,']


KeyboardInterrupt: 

In [ ]:
Audio("output.wav")

In [ ]:
import os
from pydub import AudioSegment

# Función para leer el archivo txt
def read_text_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Dividir el texto por párrafos
def split_text_by_paragraphs(text):
    # Dividir el texto por doble salto de línea que indica párrafos
    paragraphs = text.split('\n\n')
    return [p for p in paragraphs if p.strip()]  # Eliminar párrafos vacíos

# Generar audios para cada párrafo
def synthesize_segments(segments, speaker_wav, language, output_prefix):
    # Convertir los segmentos (párrafos) a audio
    file_paths = []
    for i, segment in enumerate(segments):
        file_path = f"{output_prefix}_part_{i}.wav"
        tts.tts_to_file(text=segment, file_path=file_path, speaker_wav=speaker_wav, language=language)
        file_paths.append(file_path)
    return file_paths

# Combinar los archivos de audio en uno solo
def combine_audio(segments, output_file, output_prefix):
    combined = AudioSegment.silent(duration=0)
    for i in range(len(segments)):
        audio = AudioSegment.from_wav(f"{output_prefix}_part_{i}.wav")
        combined += audio
    combined.export(output_file, format='wav')

# Limpiar archivos de audio temporales
def clean_up(file_paths):
    for file_path in file_paths:
        os.remove(file_path)

# Uso del código
file_path = '/content/storie.txt'  # Elige el archivo .txt que quieras procesar
speaker_wav = "/content/fer.wav"
language = "es"
output_prefix = "output"
final_output = "output.wav"

# Leer el contenido del archivo
texto = read_text_from_file(file_path)

# Dividir el texto por párrafos
segments = split_text_by_paragraphs(texto)

# Sintetizar audio para cada segmento (párrafo)
file_paths = synthesize_segments(segments, speaker_wav, language, output_prefix)

# Combinar los audios en un solo archivo
combine_audio(segments, final_output, output_prefix)

# Limpiar archivos temporales
clean_up(file_paths)


In [ ]:
Audio("output.wav")

In [ ]:
import os
from pydub import AudioSegment

# Función para leer el archivo txt
def read_text_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Dividir el texto por párrafos
def split_text_by_paragraphs(text):
    # Dividir el texto por doble salto de línea que indica párrafos
    paragraphs = text.split('\n\n')
    return [p for p in paragraphs if p.strip()]  # Eliminar párrafos vacíos

# Generar audios para cada párrafo
def synthesize_segments(segments, speaker_wav, language, output_prefix):
    # Convertir los segmentos (párrafos) a audio
    file_paths = []
    for i, segment in enumerate(segments):
        file_path = f"{output_prefix}_part_{i}.wav"
        tts.tts_to_file(text=segment, file_path=file_path, speaker_wav=speaker_wav, language=language)
        file_paths.append(file_path)
    return file_paths



# Limpiar archivos de audio temporales
def clean_up(file_paths):
    for file_path in file_paths:
        os.remove(file_path)

# Uso del código
file_path = '/content/historia.txt'  # Elige el archivo .txt que quieras procesar
speaker_wav = "/content/vianey.wav"
language = "es"
output_prefix = "output"
final_output = "output.wav"

# Leer el contenido del archivo
texto = read_text_from_file(file_path)

# Dividir el texto por párrafos
segments = split_text_by_paragraphs(texto)

# Sintetizar audio para cada segmento (párrafo)
file_paths = synthesize_segments(segments, speaker_wav, language, output_prefix)

# Combinar los audios en un solo archivo
combine_audio(segments, final_output, output_prefix)

# Limpiar archivos temporales
clean_up(file_paths)


# ***NECESARIO***

In [7]:
import os
import json
from pydub import AudioSegment

# Funciones para el proceso

def read_json_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# Extraer segmentos de texto del JSON
# Separamos los diálogos por personaje y el narrador
def extract_segments_from_json(data):
    scenes = data.get("escenas", [])
    segments = []
    for scene in scenes:
        for speaker, text in scene.items():
            segments.append((speaker, text))
    return segments

# Generar audios para cada segmento (narrador y personajes)
def synthesize_segments(segments, speaker_wav, language, output_prefix):
    file_paths = []
    for i, (speaker, segment) in enumerate(segments):
        file_path = f"{output_prefix}_part_{i}.wav"
        tts.tts_to_file(text=segment, file_path=file_path, speaker_wav=speaker_wav, language=language)
        file_paths.append((speaker, file_path))
    return file_paths



# Limpiar archivos de audio temporales
def clean_up(file_paths):
    for _, file_path in file_paths:
        os.remove(file_path)

# Uso del código
file_path = '/content/historia_20241020_110010.json'  # Elige el archivo .json que quieras procesar
speaker_wav = "/content/fer.wav"
language = "es"
output_prefix = "output"
final_output = "output.wav"

# Leer el contenido del archivo JSON
data = read_json_from_file(file_path)

# Extraer los segmentos de texto del JSON
segments = extract_segments_from_json(data)

# Sintetizar audio para cada segmento (narrador y personajes)
file_paths = synthesize_segments(segments, speaker_wav, language, output_prefix)

# Combinar los audios en un solo archivo
combine_audio(segments, final_output, output_prefix)

# Limpiar archivos temporales
clean_up(file_paths)


In [8]:
import os
import json
from pydub import AudioSegment
import coqui_tts as tts

# Funciones para el proceso

def read_json_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# Extraer el campo 'historia' del JSON y dividir en oraciones por los puntos
def extract_segments_from_json(data):
    historia = data.get("historia", "")
    if historia:
        # Dividimos la historia por puntos y eliminamos espacios extra
        oraciones = [oracion.strip() for oracion in historia.split('.') if oracion.strip()]
        return [("narrador", oracion + '.') for oracion in oraciones]  # Añadimos el punto de nuevo al final de cada oración
    return []

# Generar audios para cada segmento (narrador)
def synthesize_segments(segments, speaker_wav, language, output_prefix):
    file_paths = []
    for i, (speaker, segment) in enumerate(segments):
        file_path = f"{output_prefix}_part_{i}.wav"
        tts.tts_to_file(text=segment, file_path=file_path, speaker_wav=speaker_wav, language=language)
        file_paths.append((speaker, file_path))
    return file_paths

# Combinar los audios en un solo archivo
def combine_audio(file_paths, final_output):
    combined = AudioSegment.empty()
    for _, file_path in file_paths:
        audio_segment = AudioSegment.from_wav(file_path)
        combined += audio_segment
    combined.export(final_output, format="wav")

# Limpiar archivos de audio temporales
def clean_up(file_paths):
    for _, file_path in file_paths:
        os.remove(file_path)

# Uso del código
file_path = '/content/historia_20241020_110010.json'  # Elige el archivo .json que quieras procesar
speaker_wav = "/content/fer.wav"
language = "es"
output_prefix = "output"
final_output = "output.wav"

# Leer el contenido del archivo JSON
data = read_json_from_file(file_path)

# Extraer los segmentos de texto del JSON (cortado por puntos)
segments = extract_segments_from_json(data)

# Sintetizar audio para cada segmento (narrador)
file_paths = synthesize_segments(segments, speaker_wav, language, output_prefix)

# Combinar los audios en un solo archivo
combine_audio(file_paths, final_output)

# Limpiar archivos temporales
clean_up(file_paths)


ModuleNotFoundError: No module named 'coqui_tts'

In [9]:
import os
import json
from pydub import AudioSegment

def read_json_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

def extract_segments_from_json(data):
    historia = data.get("historia", "")
    # Dividir el texto en oraciones
    sentences = historia.split('.')
    # Filtrar oraciones vacías y agregar el punto final de nuevo
    segments = [sentence.strip() + '.' for sentence in sentences if sentence.strip()]
    return segments

def synthesize_segments(segments, speaker_wav, language, output_prefix):
    file_paths = []
    for i, segment in enumerate(segments):
        file_path = f"{output_prefix}_part_{i}.wav"
        tts.tts_to_file(text=segment, file_path=file_path, speaker_wav=speaker_wav, language=language)
        file_paths.append(file_path)
    return file_paths

def combine_audio(file_paths, final_output):
    combined = AudioSegment.empty()
    for file_path in file_paths:
        audio = AudioSegment.from_wav(file_path)
        combined += audio
    combined.export(final_output, format="wav")

def clean_up(file_paths):
    for file_path in file_paths:
        os.remove(file_path)

# Uso del código
file_path = '/content/historia_20241020_110010.json'
speaker_wav = "/content/fer.wav"
language = "es"
output_prefix = "output"
final_output = "output.wav"

# Leer el contenido del archivo JSON
data = read_json_from_file(file_path)

# Extraer los segmentos de texto del JSON
segments = extract_segments_from_json(data)

# Sintetizar audio para cada segmento
file_paths = synthesize_segments(segments, speaker_wav, language, output_prefix)

# Combinar los audios en un solo archivo
combine_audio(file_paths, final_output)

# Limpiar archivos temporales
clean_up(file_paths)


 > Text splitted to sentences.
['Luna Verde era una joven con el cabello verde brillante y ojos que brillaban como las estrellas en la noche.']
 > Processing time: 6.017137050628662
 > Real-time factor: 0.7177674195358457
 > Text splitted to sentences.
['Tenía una conexión especial con la naturaleza, y podía comunicarse con las plantas y animales del bosque.']
 > Processing time: 2.83764386177063
 > Real-time factor: 0.35573801027950963
 > Text splitted to sentences.
['Vivía en un pequeño pueblo rodeado de una densa selva, donde se criaban árboles altos y frondosos.']
 > Processing time: 3.3401012420654297
 > Real-time factor: 0.3780140448568137
 > Text splitted to sentences.
['Cada mañana, Luna salía a caminar por el bosque, y se sentía en paz entre las ramas y las hojas que susurraban sus secretos.']
 > Processing time: 4.40251088142395
 > Real-time factor: 0.44451683701827105
 > Text splitted to sentences.
['Un día, mientras exploraba el bosque, Luna se encontró con un grupo de árbo

# ***NO NECESARIO, COMPRIME AUDIOS EN ZIP***

In [ ]:
import os
import json
import zipfile
from pydub import AudioSegment

# Funciones para el proceso

def read_json_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# Extraer segmentos de texto del JSON
# Separamos los diálogos por personaje y el narrador
def extract_segments_from_json(data):
    scenes = data.get("escenas", [])
    segments = []
    for scene in scenes:
        for speaker, text in scene.items():
            segments.append((speaker, text))
    return segments

# Generar audios para cada segmento (narrador y personajes)
def synthesize_segments(segments, speaker_wav, language, output_prefix, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    file_paths = []
    for i, (speaker, segment) in enumerate(segments):
        file_path = os.path.join(output_folder, f"{output_prefix}_part_{i}.wav")
        tts.tts_to_file(text=segment, file_path=file_path, speaker_wav=speaker_wav, language=language)
        file_paths.append((speaker, file_path))
    return file_paths

# Crear un archivo ZIP con los audios generados
def create_zip_from_folder(folder_path, zip_name):
    with zipfile.ZipFile(zip_name, 'w') as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file), arcname=file)

# Limpiar archivos de audio temporales
def clean_up(file_paths):
    for _, file_path in file_paths:
        os.remove(file_path)

# Uso del código
file_path = '/content/ejemplo.json'  # Elige el archivo .json que quieras procesar
speaker_wav = "/content/fer.wav"
language = "es"
output_prefix = "output"
output_folder = "audio_parts"
zip_output = "audio_parts.zip"

# Leer el contenido del archivo JSON
data = read_json_from_file(file_path)

# Extraer los segmentos de texto del JSON
segments = extract_segments_from_json(data)

# Sintetizar audio para cada segmento (narrador y personajes)
file_paths = synthesize_segments(segments, speaker_wav, language, output_prefix, output_folder)

# Crear un archivo ZIP con los audios generados
create_zip_from_folder(output_folder, zip_output)

# Limpiar archivos temporales
clean_up(file_paths)


 > Text splitted to sentences.
['En un bosque frondoso y vibrante, donde la luz del sol se filtraba entre las hojas, vivía un oso llamado Bruno.', 'A diferencia de los otros osos, Bruno tenía un corazón lleno de frustración.', 'Su hogar, una cueva en la ladera de una montaña, se había vuelto un lugar oscuro y solitario, y él creía que si podía hacer que el bosque ardiera, todos tendrían que abandonarlo, dejando solo su cueva para él.']
 > Processing time: 12.571494579315186
 > Real-time factor: 0.4254374915187668
 > Text splitted to sentences.
['¡Si el bosque se quema, finalmente tendré la paz que deseo!', '¡Nadie podrá molestarme!']
 > Processing time: 2.8523590564727783
 > Real-time factor: 0.3721745242095766
 > Text splitted to sentences.
['Pero lo que Bruno no sabía era que su plan estaba a punto de desatar una serie de eventos que cambiarían su vida y la de todos los animales del bosque.', 'Mientras tanto, en el cielo, volaba un conejo llamado Ray, un auténtico superhéroe.', 'Con 